# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Optimization-in-Julia" data-toc-modified-id="Optimization-in-Julia-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Optimization in Julia</a></div><div class="lev2 toc-item"><a href="#Flowchart" data-toc-modified-id="Flowchart-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Flowchart</a></div><div class="lev2 toc-item"><a href="#DCP-Using-Convex.jl" data-toc-modified-id="DCP-Using-Convex.jl-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>DCP Using Convex.jl</a></div><div class="lev3 toc-item"><a href="#Example:-microbiome-regression-analysis" data-toc-modified-id="Example:-microbiome-regression-analysis-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Example: microbiome regression analysis</a></div><div class="lev3 toc-item"><a href="#Sum-to-zero-regression" data-toc-modified-id="Sum-to-zero-regression-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Sum-to-zero regression</a></div><div class="lev3 toc-item"><a href="#Sum-to-zero-lasso" data-toc-modified-id="Sum-to-zero-lasso-123"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Sum-to-zero lasso</a></div><div class="lev3 toc-item"><a href="#Sum-to-zero-group-lasso" data-toc-modified-id="Sum-to-zero-group-lasso-124"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Sum-to-zero group lasso</a></div><div class="lev3 toc-item"><a href="#Example:-matrix-completion" data-toc-modified-id="Example:-matrix-completion-125"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>Example: matrix completion</a></div><div class="lev2 toc-item"><a href="#Nonlinear-programming-(NLP)" data-toc-modified-id="Nonlinear-programming-(NLP)-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Nonlinear programming (NLP)</a></div>

# Optimization in Julia

This lecture gives an overview of some optimization tools in Julia.

**Warning**: Some code chunks will not run on the server because Mosek/Gurobi (commercial software) are not available on the server.

In [ ]:
versioninfo()

## Flowchart

* Statisticians do optimizations in daily life: maximum likelihood estimation, machine learning, ...

* Category of optimization problems:

    1. Problems with analytical solutions: least squares, principle component analysis, canonical correlation analysis, ...
    
    2. Problems subject to Disciplined Convex Programming (DCP): linear programming (LP), quadratic programming (QP), second-order cone programming (SOCP), semidefinite programming (SDP), and geometric programming (GP).
    
    3. Nonlinear programming (NLP): Newton type algorithms, Fisher scoring algorithm, EM algorithm, MM algorithms. 
    
    4. Large scale optimization: ADMM, SGD, ...
    
![Flowchart](./optimization_flowchart.png)    

## DCP Using Convex.jl

Standard convex problem classes like LP (linear programming), QP (quadratic programming), SOCP (second-order cone programming), SDP (semidefinite programming), and GP (geometric programming), are becoming a **technology**.

![DCP Hierarchy](./convex-hierarchy.png)

Getting familiar with **good** optimization softwares broadens the scope and scale of problems we are able to solve in statistics. Following table lists some of the best optimization softwares. 

|           |   | LP | MILP | SOCP |     MISOCP     | SDP | GP | NLP | MINLP |   | R | Matlab | Julia | Python |   | Cost |
|:---------:|:-:|:--:|:----:|:----:|:--------------:|:---:|:--:|:---:|:-----:|:-:|:-:|:------:|:-----:|:------:|:-:|:----:|
|   **modeling tools**   |   |    |      |      |                |     |    |     |       |   |   |        |       |        |   |      |
|    cvx    |   |  x |   x  |   x  |        x       |  x  |  x |     |       |   |   |    x   |       |    x   |   |   A  |
| Convex.jl |   |  x |   x  |   x  |        x       |  x  |    |     |       |   |   |        |   x   |        |   |   O  |
|  JuMP.jl  |   |  x |   x  |   x  |        x       |     |    |  x  |   x   |   |   |        |   x   |        |   |   O  |
|   **convex solvers** |   |    |      |      |                |     |    |     |       |   |   |        |       |        |   |      |
|   Mosek   |   |  x |   x  |   x  |        x       |  x  |  x |  x  |       |   | x |    x   |   x   |    x   |   |   A  |
|   Gurobi  |   |  x |   x  |   x  |        x       |     |    |     |       |   | x |    x   |   x   |    x   |   |   A  |
|   CPLEX   |   |  x |   x  |   x  |        x       |     |    |     |       |   | x |    x   |   x   |    x   |   |   A  |
|    SCS    |   |  x |      |   x  |                |  x  |    |     |       |   |   |    x   |   x   |    x   |   |   O  |
|   SeDuMi  |   |  x |      |   x  |                |  x  |  ? |     |       |   |   |    x   |       |        |   |   O  |
|   SDPT3   |   |  x |      |   x  |                |  x  |  ? |     |       |   |   |    x   |       |        |   |   O  |
|   **NLP solvers**  |   |    |      |      |                |     |    |     |       |   |   |        |       |        |   |      |
|   NLopt   |   |  x |      |      |                |     |    |  x  |       |   |   |    x   |   x   |    x   |   |   O  |
|   Ipopt   |   |  x |      |      |                |     |    |  x  |       |   |   |    x   |   x   |    x   |   |   O  |
|   KNITRO  |   |  x |   x  |      |                |     |    |  x  |   x   |   | x |    x   |   x   |    x   |   |   $  |

* O: open source  
* A: free academic license  
* $: commercial

* Difference between **modeling tool** and **solvers**

    - **Modeling tools** such as cvx (for Matlab) and Convex.jl (Julia analog of cvx) implement the disciplined convex programming (DCP) paradigm proposed by Grant and Boyd (2008) <http://stanford.edu/~boyd/papers/disc_cvx_prog.html>. DCP prescribes a set of simple rules from which users can construct convex optimization problems easily.
    
    - **Solvers** (Mosek, Gurobi, Cplex, SCS, ...) are concrete software implementation of optimization algorithms. My favorite ones are: Mosek/Gurobi/SCS for DCP and Ipopt/NLopt for nonlinear programming. Mosek and Gurobi are commercial software but free for academic use. SCS/Ipopt/NLopt are open source.  
    
    - Modeling tools usually have the capability to use a variety of solvers. But modeling tools are solver agnostic so users do not have to worry about specific solver interface.

### Example: microbiome regression analysis

We illustrate optimization tools in Julia using microbiome analysis as an example.

16S microbiome sequencing techonology generates sequence counts of various organisms (OTUs, operational taxonomic units) in samples. 

![Microbiome Data](./microbiome_data.png)

For statistical analysis, counts are normalized into **proportions** for each sample, resulting in a covariate matrix $\mathbf{X}$ with all rows summing to 1. For identifiability, we need to add a sum-to-zero constraint to the regression cofficients. In other words, we need to solve a **constrained least squares problem**  
$$
    \text{minimize} \frac{1}{2} \|\mathbf{y} - \mathbf{X} \beta\|_2^2
$$
subject to the constraint $\sum_{j=1}^p \beta_j = 0$. For simplicity we ignore intercept and non-OTU covariates in this presentation.

Let's first generate an artifical data set.

In [ ]:
srand(123) # seed

n, p = 100, 50
X = rand(n, p)
X = Diagonal(1 ./ vec(sum(X, 2))) * X
β = sprandn(p, 0.1) # sparse vector with about 10% non-zero entries
y = X * β + randn(n);

### Sum-to-zero regression

The sum-to-zero contrained least squares is a standard quadratic programming (QP) problem so should be solved easily by any QP solver.

Load the Convex.jl package and chooose a solver. Note Gurobi and Mosek (commercial) are not installed on the server; use SCS solver (open source) on the server.

In [ ]:
using Convex

## Use Gurobi solver
#using Gurobi
#solver = GurobiSolver(OutputFlag=0)
#set_default_solver(solver)

# # Use Mosek solver
# using Mosek
# solver = MosekSolver(LOG=1)
# set_default_solver(solver)

# Use SCS solver
using SCS
solver = SCSSolver(verbose=1)
set_default_solver(solver)

In [ ]:
β̂cls = Variable(size(X, 2))
problem = minimize(0.5sumsquares(y - X * β̂cls)) # objective
problem.constraints += sum(β̂cls) == 0 # constraint
@time solve!(problem)

### Sum-to-zero lasso

Suppose we want to know which organisms (OTU) are associated with the response. We can answer this question using a sum-to-zero contrained lasso
$$
    \text{minimize} \frac 12 \|\mathbf{y} - \mathbf{X} \beta\|_2^2 + \lambda \|\beta\|_1
$$
subject to the constraint $\sum_{j=1}^p \beta_j = 0$. Varying $\lambda$ from small to large values will generate a solution path.

In [ ]:
# Use SCS solver
using SCS
solver = SCSSolver(verbose=0)
set_default_solver(solver)

# solve at a grid of λ
λgrid = 0:0.01:0.35
β̂path = zeros(length(λgrid), size(X, 2)) # each row is β̂ at a λ
β̂classo = Variable(size(X, 2))
@time for i in 1:length(λgrid)
    λ = λgrid[i]
    # objective
    problem = minimize(0.5sumsquares(y - X * β̂classo) + λ * sum(abs, β̂classo))
    # constraint
    problem.constraints += sum(β̂classo) == 0 # constraint
    solve!(problem)
    β̂path[i, :] = β̂classo.value
end

In [ ]:
using Plots; gr()
using LaTeXStrings

p = plot(collect(λgrid), β̂path, legend=:none)
xlabel!(p, L"\lambda")
ylabel!(p, L"\hat \beta")
title!(p, "Sum-to-Zero Lasso")

### Sum-to-zero group lasso

Suppose we want to do variable selection not at the OTU level, but at the Phylum level. OTUs are clustered into various Phyla. We can answer this question using a sum-to-zero contrained group lasso
$$
    \text{minimize} \frac 12 \|\mathbf{y} - \mathbf{X} \beta\|_2^2 + \lambda \sum_j \|\mathbf{\beta}_j\|_2
$$
subject to the constraint $\sum_{j=1}^p \beta_j = 0$, where $\mathbf{\beta}_j$ are regression coefficients corresponding to the $j$-th phylum. This is a second-order cone programming (SOCP) problem readily modeled by Convex.jl.

Let's assume each 10 continuous OTUs belong to one Phylum.

In [ ]:
# Use SCS solver
using SCS
solver = SCSSolver(verbose=0)
set_default_solver(solver)

# solve at a grid of λ
λgrid = 0.1:0.005:0.5
β̂pathgrp = zeros(length(λgrid), size(X, 2)) # each row is β̂ at a λ
β̂classo = Variable(size(X, 2))
@time for i in 1:length(λgrid)
    λ = λgrid[i]
    # objective
    obj = 0.5sumsquares(y - X * β̂classo)
    for j in 1:(size(X, 2)/10)
        βj = β̂classo[(10(j-1)+1):10j]
        obj += λ * vecnorm(βj)
    end
    problem = minimize(obj)
    # constraint
    problem.constraints += sum(β̂classo) == 0 # constraint
    solve!(problem)
    β̂pathgrp[i, :] = β̂classo.value
end

In [ ]:
p2 = plot(collect(λgrid), β̂pathgrp, legend=:none)
xlabel!(p2, L"\lambda")
ylabel!(p2, L"\hat \beta")
title!(p2, "Sum-to-Zero Group Lasso")

### Example: matrix completion

Load the $128 \times 128$ Lena picture with missing pixels.

In [ ]:
using Images

lena = load("lena128missing.png")

In [ ]:
# convert to real matrices
Y = Float64.(lena)

We fill out the missin pixels uisng a **matrix completion** technique developed by Candes and Tao
$$
    \text{minimize } \|\mathbf{X}\|_*
$$
$$
    \text{subject to } x_{ij} = y_{ij} \text{ for all observed entries } (i, j).
$$
Here $\|\mathbf{M}\|_* = \sum_i \sigma_i(\mathbf{M})$ is the nuclear norm. In words we seek the matrix with minimal nuclear norm that agrees with the observed entries. This is a semidefinite programming (SDP) problem readily modeled by Convex.jl.

This example takes long because of high dimensionality. **Don't run on server**

In [ ]:
# Use Mosek solver
using Mosek
solver = MosekSolver(LOG=1)
set_default_solver(solver)

# Linear indices of obs. entries
obsidx = find(Y .≠ 0.0)
# Create optimization variables
X = Convex.Variable(size(Y))
# Set up optmization problem
problem = minimize(nuclearnorm(X))
problem.constraints += X[obsidx] == Y[obsidx]
# Solve the problem by calling solve
@time solve!(problem)

In [ ]:
colorview(Gray, X.value)

## Nonlinear programming (NLP)

We use MLE of Gamma distribution to illustrate some rudiments of nonlinear programming (NLP) in Julia. 

Let $x_1,\ldots,x_m$ be a random sample from the gamma density
$$
f(x) = \Gamma(\alpha)^{-1} \beta^{\alpha} x^{\alpha-1} e^{-\beta x}
$$
on $(0,\infty)$. The loglikelihood function is
$$
    L(\alpha, \beta) = m [- \ln \Gamma(\alpha) + \alpha \ln \beta + (\alpha - 1)\overline{\ln x} - \beta \bar x],
$$
where $\overline{x} = \frac{1}{m} \sum_{i=1}^m x_i$ and 
$\overline{\ln x} = \frac{1}{m} \sum_{i=1}^m \ln x_i$. Following function 
evaluates the log-pdf of one data point `x` at parameter `α` and `β`.

In [ ]:
function gamma_logpdf(x::Vector, α::Real, β::Real)
    m = length(x)
    avg = mean(x)
    logavg = sum(log, x) / m
    m * (- lgamma(α) + α * log(β) + (α - 1) * logavg - β * avg)
end
x = rand(5)
gamma_logpdf(x, 1.0, 1.0)

Many optimization algorithms involve taking derivatives of the objective function. The `ForwardDiff` package implements automatic differentiation. For example, to compute the derivative and Hessian of the log-likelihood with data `x` at `α=1.0` and `β=1.0`.

In [ ]:
using ForwardDiff
ForwardDiff.gradient(r -> gamma_logpdf(x, r...), [1.0; 1.0])

In [ ]:
ForwardDiff.hessian(r -> gamma_logpdf(x, r...), [1.0; 1.0])

Generate data:

In [ ]:
using Distributions

srand(123)
(n, p) = (1000, 2)
(α, β) = 5.0 * rand(p)
x = rand(Gamma(α, β), n)
println("True parameter values:")
println("α = ", α, ", β = ", β)

We use JuMP.jl to define and solve our NLP problem.

In [ ]:
using JuMP, Ipopt, NLopt

m = Model(solver = IpoptSolver())
# m = Model(solver = NLoptSolver(algorithm=:LD_MMA))

myf(a, b) = gamma_logpdf(x, a, b)
JuMP.register(m, :myf, 2, myf, autodiff=true)
@variable(m, α >= 1e-8)
@variable(m, β >= 1e-8)
@NLobjective(m, Max, myf(α, β))

print(m)
status = solve(m)

println("MLE (JuMP):")
println("α = ", α, ", β = ", β)
println("Objective value: ", getobjectivevalue(m))
println("α = ", getvalue(α), ", β = ", 1 / getvalue(β))
println("MLE (Distribution package):")
println(fit_mle(Gamma, x))